In [1]:
# Clone the Anomalib repository from GitHub and Install Anomalib
# Install the Lightning library for training
# Install Kornia for image processing
# Install FrEIA for normalizing flows
# Install python-dotenv for environment variable management
# Install OpenCLIP for contrastive learning
!git clone https://github.com/openvinotoolkit/anomalib.git
%cd anomalib
%pip install .
!pip install lightning
!pip install kornia
!pip install git+https://github.com/VLL-HD/FrEIA.git
!pip install python-dotenv
!pip install open_clip_torch

fatal: destination path 'anomalib' already exists and is not an empty directory.
/content/anomalib
Processing /content/anomalib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for anomalib: filename=anomalib-2.0.0.dev0-py3-none-any.whl size=486692 sha256=804e974c5aee9be0dd0f6119bfd52633c3e002565fbe9b3332461f1d9b981dc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ndn1846d/wheels/d2/67/5a/24537c94d996a7c04fc11b75dee04161307ba49c2fac3a9bcc
Successfully built anomalib
  Attempting uninstall: anomalib
    Found existing installation: anomalib 2.0.0.dev0
    Uninstalling anomalib-2.0.0.dev0:
      Successfully uninstalled anomalib-2.0.0.dev0
  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached torchmetrics-1.5.1-py3-none-any.whl.metadata (20 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
Using cached lightning-2.4.0-py3-none-any.

In [2]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded
#   to this directory.
dataset_root = Path.cwd().parent / "datasets" / "MVTec"

In [3]:
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader

from anomalib import TaskType
from anomalib.data import MVTec, PredictDataset
from anomalib.engine import Engine
from anomalib.models import Patchcore, EfficientAd
from anomalib.utils.post_processing import superimpose_anomaly_map

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


# Load the Dataset


In [13]:
# Task specification to segmentation
task = TaskType.SEGMENTATION

In [14]:
import matplotlib.pyplot as plt
# Initialize the datamodule for the specified category (tile, leather, grid)
categories = ["tile", "leather", "grid"]
for category in categories:
    print(f"Training on category: {category}")

    # Create the datamodule for the current category
    datamodule = MVTec(
        root=dataset_root,
        category=category,
        image_size=(64, 64),
        train_batch_size=16,
        eval_batch_size=16,
        num_workers=4,
        task=task,
    )



Training on category: tile
Training on category: leather
Training on category: grid


# **Initialize PathCore model and Training Engine**
- Backbone specifies whoch pretrained network to use as the feature extractor and ResNet-18 is ood for CNN architecture
- Specify that the training perfromance will be measured using AUROC metric which works well for anomaly detection


In [6]:
 # Initialize the PatchCore model
model = Patchcore(backbone="resnet18")

# Setup callbacks for training
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
        dirpath=f"checkpoints/{category}",
        filename="{epoch:02d}-{pixel_AUROC:.2f}",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=3,
    ),
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [7]:
# Initialize the training engine
engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto",
    devices=1,
    logger=False,
)

# Train the model
engine.fit(datamodule=datamodule, model=model)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
mvtec: 5.26GB [04:09, 21.1MB/s]                            
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
INFO: 
  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 2.8 M  | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_metrics | MetricCollection         | 0      | train
3 | image_threshold       | F1Adapt

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/optimization/automatic.py:132: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]



Selecting Coreset Indices.:   0%|          | 0/1293 [00:00<?, ?it/s]

Selecting Coreset Indices.:   0%|          | 2/1293 [00:00<01:17, 16.64it/s]

Selecting Coreset Indices.:   0%|          | 5/1293 [00:00<00:55, 23.19it/s]

Selecting Coreset Indices.:   1%|          | 10/1293 [00:00<00:37, 33.82it/s]

Selecting Coreset Indices.:   1%|▏         | 17/1293 [00:00<00:27, 46.32it/s]

Selecting Coreset Indices.:   2%|▏         | 23/1293 [00:00<00:25, 50.58it/s]

Selecting Coreset Indices.:   2%|▏         | 29/1293 [00:00<00:24, 52.66it/s]

Selecting Coreset Indices.:   3%|▎         | 35/1293 [00:00<00:26, 47.57it/s]

Selecting Coreset Indices.:   3%|▎         | 40/1293 [00:00<00:26, 47.65it/s]

Selecting Coreset Indices.:   4%|▎         | 46/1293 [00:01<00:25, 49.35it/s]

Selecting Coreset Indices.:   4%|▍         | 51/1293 [00:01<00:25, 48.11it/s]

Selecting Coreset Indices.:   4%|▍         | 56/1293 [00:01<00:26, 45.92it/s]

Selecting Coreset Indices.:   5%|▍         | 61/1293 [00:01<0

In [8]:
# Test the model
engine.test(datamodule=datamodule, model=model)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.5271511673927307     │
│       image_F1Score       │    0.8421052694320679     │
│        pixel_AUROC        │    0.7455759644508362     │
└───────────────────────────┴───────────────────────────┘

[{'pixel_AUROC': 0.7455759644508362,
  'image_AUROC': 0.5271511673927307,
  'image_F1Score': 0.8421052694320679}]

In [9]:
 # Initialize the EfficientAD
model2 = EfficientAd(
    imagenet_dir="./path/to/imagenette",
    teacher_out_channels=384,
    lr=0.001,  # Adjusted learning rate
    weight_decay=0.00001,
)

# Initialize the datamodule for the specified category (tile, leather, grid)
categories = ["tile", "leather", "grid"]
for category in categories:
    print(f"Training on category: {category}")

    # Create the datamodule for the current category
    datamodule2 = MVTec(
      root=dataset_root,
      category=category,
      image_size=(312, 312),
      train_batch_size=1,
      eval_batch_size=4,
      num_workers=8,
      task=task,
    )

# Setup callbacks for training
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
        dirpath=f"checkpoints/{category}",
        filename="{epoch:02d}-{pixel_AUROC:.2f}",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=3,
    ),
]

Training on category: tile
Training on category: leather
Training on category: grid


In [ ]:
# Initialize the training engine
from lightning.pytorch.loggers import TensorBoardLogger
logger = TensorBoardLogger("logs", name=category)
engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="cpu",
    devices=1,
    logger=False,
    max_epochs=50
)
# Train the model for EfficientAdModel
engine.fit(datamodule=datamodule2, model=model2)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /content/anomalib/checkpoints/grid exists and is not empty.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
IN

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/lightning_model.py:98: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/usr/local/lib/python3.10/dist-packages/torchv

In [ ]:
# Test the EfficientAd model
engine.test(datamodule=datamodule, model=model)